In [1]:
# use a baseline trained only with PPL, which has already converged and will be used as our baseline.
# they do teacher forcing all the way.

In [2]:
from data_generators import get_iterator, get_dataset
from classifiers import theme_classifier

In [3]:
import torch
import torch.nn as nn

In [4]:
from torchtext.vocab import GloVe 
GLOVE_EMBEDDING = GloVe(name="6B", dim=300)

In [5]:
train_dataset, val_dataset, test_dataset, review_text_FIELD, theme_FIELD = get_dataset(vectors = GLOVE_EMBEDDING)

In [6]:
batch_size = 20
train_iter = get_iterator(train_dataset, batch_size, train=True, shuffle=True, repeat=False)
val_iter = get_iterator(val_dataset, batch_size, train=False, shuffle=True, repeat=False)
test_iter = get_iterator(test_dataset, batch_size, train=False, shuffle=True, repeat=False)

In [7]:
val_list = list(val_iter)

In [8]:
batch = val_list[50]
x = batch.review_text.transpose(1, 0).int()[:10]
y = batch.theme.int()

for idx in range(x.shape[0]):
    #print(x.shape, y.shape)
    print("{} | {}".format(' '.join([train_dataset.fields['review_text'].vocab.itos[_] for _ in x[idx]]),
         train_dataset.fields['theme'].vocab.itos[y[idx]]))

<sos> awesome sauce <eos> | other
<sos> totally recommend <eos> | other
<sos> <unk> . <eos> | other
<sos> truly amazing <eos> | other
<sos> talky . <eos> | other
<sos> must see <eos> | other
<sos> 3.5 5 <eos> | other
<sos> 5 stars <eos> | other
<sos> movie ? <eos> | other
<sos> brutal . <eos> | other


In [9]:
theme_FIELD.vocab.itos

['<unk>', 'other', 'plot', 'acting', 'effect', 'production']

In [10]:
vocab_size = review_text_FIELD.vocab.vectors.shape[0]
label_size = len(theme_FIELD.vocab) - 1
emb_dim = review_text_FIELD.vocab.vectors.shape[1]
vectors = train_dataset.fields["review_text"].vocab.vectors
hidden_dim = 1024
layers = 2
dropout = .5

vocab_size, label_size, emb_dim, vectors.shape

(12304, 5, 300, torch.Size([12304, 300]))

In [11]:
from baseline_model import BaseModel, repackage_hidden

In [12]:
import numpy as np
import gc

In [13]:
EOS_token = review_text_FIELD.vocab.stoi['<eos>']
EOS_token

3

In [14]:
def forward_pass(model, data, labels, i):
    
    split_tf = data.shape[0] - (i % data.shape[0])
    #print(split_tf)
    
    seq_len = data.shape[0]
    data_tf = data[:split_tf+int(i==0),:]
    data_nf = data[split_tf:,:]

    output_flat = None
    hidden = None
    
    if split_tf > 0:

        data = data_tf
        output_tf, hidden = model(data, labels, hidden)
        repackage_hidden(hidden)
        output_flat = output_tf.contiguous().view(-1, vocab_size)

    if split_tf < seq_len:

        data = data_nf
        shape = tuple((*data_nf.shape, vocab_size))
        output_nf = torch.zeros(shape).cuda()
        hidden_i = None
        data_i = data[0,:]    

        for di in range(data_nf.shape[0]):
            params = data_i.unsqueeze(0), labels, hidden_i
            output_i, hidden_i = model(*params)
            hidden_i = repackage_hidden(hidden_i)
            topv, topi = output_i.topk(1)
            data_i = topi.squeeze().detach()
            output_nf[di,:] = output_i

        temp_output_flat = output_nf.contiguous().view(-1, vocab_size)
        if output_flat is None:
            output_flat = temp_output_flat
        else:
            output_flat = torch.cat([output_flat, temp_output_flat], 0)
            
    return output_flat

In [15]:
def transtaltion2string(raw_translations, target_vocab = review_text_FIELD, max_words=30000):
    string_translations = []
    for raw_sentence in raw_translations:
        string_sentence = []
        for i, word_idx in enumerate(raw_sentence):
            if i == max_words: break
            word = target_vocab.vocab.itos[word_idx]
            if word != '<sos>':
                string_sentence.append(word)
        string_translations.append(string_sentence)
    return [' '.join(_) for _ in string_translations]

In [16]:
def get_predicted_labels(output_flat, original_shape, ground_truth):
    topv, topi = output_flat.topk(1, -1)
    topi = topi.view(*original_shape)
    
    predictions = []
    for i in range(topi.shape[1]):
        predicted_class = theme_classifier(transtaltion2string([topi[:,i]])[0])
        idx_class = theme_FIELD.vocab.stoi[predicted_class] - 1 
        predictions.append(idx_class)
    predictions = torch.tensor(predictions).cuda()
    classifier_output = (predictions == ground_truth).float()
    acc_score = classifier_output.mean().item()
    return classifier_output

In [17]:
def evaluate(model, data_source, criterion, teacher_forcing = False):
    model.eval()
    total_loss_e = 0
    total_number_of_words = 0
    
    with torch.no_grad():
        for i, batch in enumerate(data_source):
            labels = batch.theme.cuda().long() - 1
            batch = batch.review_text.cuda().long()
            
            if batch.shape[0] > 3:
                data, targets = batch[1:-1,:], batch[2:,:]
                target_flat = targets.contiguous().view(-1)
                
                tf = i if not teacher_forcing else 0
                output_flat = forward_pass(model, data, labels, tf)
                classifier_output = get_predicted_labels(output_flat, targets.shape, labels)
                
                batch_loss = criterion(output_flat, target_flat, classifier_output, targets.shape).detach().item()
                
                number_of_words = data.shape[0] * data.shape[1]
                total_loss_e += batch_loss * number_of_words
                total_number_of_words += number_of_words
            
    #print(total_loss_e, total_number_of_words)
    return total_loss_e / total_number_of_words

In [18]:
def train(model, ep0, epN, train_iter, dev_iter, optimizer, criterion, 
          max_grad_norm, model_name, best_ppl = float('inf'), teacher_forcing = False):
    
    best_ppl = best_ppl
    
    len_train_iter = len(train_iter)
    for epoch in range(ep0, epN):
        model.train()
        total_loss_e = 0
        total_number_of_words = 0 
        
        for i, batch in enumerate(train_iter):

            labels = batch.theme.cuda().long() - 1
            batch = batch.review_text.cuda().long()
            hidden = None
            
            if batch.shape[0] > 3:
                data, targets = batch[1:-1,:], batch[2:,:]
                
                tf = i if not teacher_forcing else 0
                output_flat = forward_pass(model, data, labels, tf)
                
                target_flat = targets.contiguous().view(-1)
                classifier_output = get_predicted_labels(output_flat, targets.shape, labels)
                batch_loss = criterion(output_flat, target_flat, classifier_output, targets.shape)
                
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
                optimizer.zero_grad()
                batch_loss.backward()
                optimizer.step()

                number_of_words = data.shape[0] * data.shape[1]
                total_loss_e += batch_loss.detach().item() * number_of_words
                total_number_of_words += number_of_words
            
                
                if i % 500 == 0:
                    cur_loss = batch_loss.detach().item() 
                    tr_ppl_print = np.exp(cur_loss)
                    print("| epoch {:3d} | batch {} / {} | train_loss {} | train_ppl {}".format(
                            epoch, i, len_train_iter, 
                            np.round(cur_loss, 3), np.round(tr_ppl_print, 3)))

                
                if i % 4999 == 1: #len_train_iter - 1:
                    cur_loss = batch_loss.detach().item()
                    tr_ppl_print = np.exp(cur_loss)
                    gc.collect()
                    val_loss_eval = evaluate(model, dev_iter, criterion_0, teacher_forcing)
                    val_ppl_print = np.exp(val_loss_eval)
                    
                    template = "| epoch {:3d} | batch {} / {} | train_loss {} | train_ppl {} | val_loss {} | val_ppl {}"
                    print(template.format(
                            epoch, i, len_train_iter, 
                            np.round(cur_loss, 3), np.round(tr_ppl_print, 3), 
                            np.round(val_loss_eval, 3), np.round(val_ppl_print, 3)))

                    if val_ppl_print < best_ppl :
                        print('old best ppl {} new best ppl {}'.format(best_ppl, val_ppl_print))
                        best_ppl = val_ppl_print
                        best_model_name = '{}_{}.model'.format(model_name, best_ppl)
                        print('save model...', best_model_name)
                        with open(best_model_name, 'wb') as file:
                            torch.save(model, file) 

                    gc.collect()
                    model.train()
                    
                if i == 40000: break

In [19]:
with open('./baseline/best_model_base_model_ppl_67.5461793533482.model', 'rb') as file:
    model = torch.load(file)
    model.eval()

In [20]:
model

BaseModel(
  (drop): Dropout(p=0.5)
  (word_embedding): Embedding(12304, 300)
  (label_embedding): Embedding(5, 20)
  (rnn): LSTM(320, 1024, num_layers=2, dropout=0.5)
  (decoder): Linear(in_features=1024, out_features=12304, bias=True)
)

In [21]:
learning_rate = 0.001

optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate)#, betas=(.9999, .9999))


def criterion(predictions, targets, labels, targets_shape):
    """
    labels: [1, 0, 0, 1, ...] 1: hit, 0: miss
    """
    c = nn.CrossEntropyLoss(reduction='none',
                           ignore_index=train_dataset.fields["review_text"].vocab.stoi['<pad>']).cuda()
    out = c(predictions, targets)
    out = (out.view(*targets_shape) * (1 - labels)).mean()
    return out

def criterion_0(predictions, targets, labels, targets_shape):
    c = nn.CrossEntropyLoss(reduction='mean',
                           ignore_index=train_dataset.fields["review_text"].vocab.stoi['<pad>']).cuda()
    out = c(predictions, targets)
    return out

def criterion_1(predictions, targets, labels, targets_shape):
    c = nn.CrossEntropyLoss(reduction='none',
                           ignore_index=train_dataset.fields["review_text"].vocab.stoi['<pad>']).cuda()
    out = c(predictions, targets)
    out = out.mean()
    return out

In [ ]:
train(model,
      ep0 = 0,
      epN = 1,
      train_iter = train_iter,
      dev_iter = val_iter,
      optimizer = optimizer,
      criterion = criterion,
      max_grad_norm = 10,
      model_name = 'no-teacher-forcing/model_ppl',
      best_ppl = float('inf'), 
      teacher_forcing = True)

| epoch   0 | batch 500 / 35138 | train_loss 0.368 | train_ppl 1.445
| epoch   0 | batch 1000 / 35138 | train_loss 0.329 | train_ppl 1.39
| epoch   0 | batch 1500 / 35138 | train_loss 0.244 | train_ppl 1.277
| epoch   0 | batch 2000 / 35138 | train_loss 0.291 | train_ppl 1.338
| epoch   0 | batch 2500 / 35138 | train_loss 1.758 | train_ppl 5.801
| epoch   0 | batch 3000 / 35138 | train_loss 0.195 | train_ppl 1.216
| epoch   0 | batch 3500 / 35138 | train_loss 0.575 | train_ppl 1.778
| epoch   0 | batch 4000 / 35138 | train_loss 0.567 | train_ppl 1.763
| epoch   0 | batch 4500 / 35138 | train_loss 0.428 | train_ppl 1.534
| epoch   0 | batch 5000 / 35138 | train_loss 0.853 | train_ppl 2.348
| epoch   0 | batch 5000 / 35138 | train_loss 0.853 | train_ppl 2.348 | val_loss 4.21 | val_ppl 67.384
old best ppl inf new best ppl 67.3838731944994
save model... no-teacher-forcing/model_ppl_67.3838731944994.model
| epoch   0 | batch 5500 / 35138 | train_loss 0.155 | train_ppl 1.167
| epoch   0 | ba

## 

In [ ]:
2+2

In [61]:
with open('./no-teacher-forcing/adam.optimizer', 'wb') as file:
    torch.save(optimizer.state_dict(), file)

In [74]:
from baseline_model import BaseModel, repackage_hidden
with open('./no-teacher-forcing/model_ppl_1.6758322557913912.model', 'rb') as file:
    model = torch.load(file)
    model.eval()

In [75]:
def criterion(predictions, targets, labels, targets_shape):
    """
    labels: [1, 0, 0, 1, ...] 1: hit, 0: miss
    """
    c = nn.CrossEntropyLoss(reduction='none',
                           ignore_index=train_dataset.fields["review_text"].vocab.stoi['<pad>']).cuda()
    out = c(predictions, targets).mean()
    return out
evaluate(model, test_iter, criterion)

6.150165933867591

In [77]:
np.exp(6.15)

468.717386782417

In [78]:
evaluate(model, test_iter, criterion_0)

6.168267680616764